#ANÁLISE DE SENTIMENTO EM REVIEWS DE TRÊS ESTRELAS: UMA PROPOSTA DE CLASSIFICAÇÃO MULTIRRÓTULO

# Seção 7. Preparação dos Dados para os Modelos de Aprendizado de Máquina

#### Importação das Bibliotecas

In [1]:
import pandas as pd
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import spacy.cli
spacy.cli.download("pt_core_news_sm")
import pt_core_news_sm
spc_pt = pt_core_news_sm.load()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carolina.gadelha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


#### Task 0: Carregar o dataset B2W-Reviews01-Anotado

In [2]:
#Carrega o dataset anotado
data = pd.read_csv('b2w-reviews-01-anotado.csv')

#### Task 01: Eliminar Reviews Neutras

In [3]:
data_anotado = data_anotado[data_anotado['anotacao'] != 'neutro']
data_anotado.reset_index(drop=True, inplace=True)
data_anotado.index.name = 'index'

In [4]:
data.anotacao.value_counts()

positivo  negativo    2971
positivo              1283
negativo               525
Name: anotacao, dtype: int64

#### Task 02: Preparação dos Dados para Modelos

In [5]:
#Pré-processamento dos textos da coluna Reviews 
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_pt = stopwords.words("portuguese")
stopwords_pt = [i for i in stopwords_pt if i not in ["não", "nenhum", "nada", "jamais", "nunca", "nem"]]

def limpa_texto(texto):
    texto = texto.lower() #Transforma texto em minúsculo
    texto = re.sub(r"[\W\d_]+", " ", texto) #Filtra apenas letras
    texto = [pal for pal in texto.split() if pal not in stopwords_pt] #Remove stopwords
    spc_texto = spc_pt(" ".join(texto))
    tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto] #Lemmatiza

    return " ".join(tokens)

#Aplica função na coluna Review
data_anotado['review'] = data_anotado['review'].apply(limpa_texto)

#### Task 03: Transformar classes de sentimento em colunas

In [ ]:
#Lista as classes de sentimento existentes no dataset
SENTIMENTOS = ['positivo','negativo']
 
#Transforma as classes em coluna com valores preenchidos 
for sentimento in SENTIMENTOS[::-1]:
    data_anotado.insert(1,sentimento,data_anotado.anotacao.apply(lambda x: 1 if sentimento in x else 0))

#### Task 04: Dividir os dados em Treino e Teste

In [ ]:
#Divide o Dataset em Treino e Teste
X = data_anotado["review"]
y = data_anotado[['positivo', 'negativo']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Task 05: Vetorizar os textos da coluna Reviews

In [ ]:
#Vetoriza as Reviews com TFIDF
vectorizer_tfidf = TfidfVectorizer(max_features=5000)
vectorizer_tfidf.fit(X)
X_train_tfidf = vectorizer_tfidf.transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

In [ ]:
#Vetoriza as Reviews com BOW
vectorizer_bow = CountVectorizer(binary=True)
vectorizer_bow.fit(X)
X_train_bow = vectorizer_bow.transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)